In [9]:
# -*- coding: utf-8 -*-
"""
Ökobilanz-Rechner für Deponiegasverwertung durch einen Dual-Fuel-Motor.

Diese Anwendung berechnet die ökologischen Auswirkungen basierend auf Nutzer-Inputs
für Gasmenge, Methangehalt und Betrachtungszeitraum.
"""

import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import webbrowser
import matplotlib.ticker as mticker

# ----------------------------------------------------------------
# GitHub Raw-URLs for your CSV files
# ----------------------------------------------------------------
URL_DUAL_FUEL_BACKGROUND = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/1.Dual_Fuel_Background_Data.csv"
URL_DUAL_FUEL_IMPACTS    = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/2.Dual_Fuel_Impacts.csv"
URL_EMISSION_IMPACTS     = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/3.Emission_Impacts.csv"
URL_ENGINE_DATA          = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/4.Engine_Data.csv"
URL_ASSUMPTIONS          = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/5.Assumptions.csv"
URL_EMISSIONS            = "https://raw.githubusercontent.com/Svenhardy/LoCaGas_App/main/Background_Data/CSV_Files/6.Emission.csv"


# ----------------------------------------------------------------
# Class for the calculation
# ----------------------------------------------------------------
class LCA_Calculator:
    def __init__(self):
        """
        Initializes the calculator and loads the CSV files directly from GitHub.
        """
        try:
            # --- Load data ---
            self.df_dual_fuel_background = pd.read_csv(URL_DUAL_FUEL_BACKGROUND, sep=";", header=[0, 1], index_col=0)
            self.df_dual_fuel_impacts = pd.read_csv(URL_DUAL_FUEL_IMPACTS, sep=";", index_col=0, skiprows=[1])
            self.df_emission_impacts = pd.read_csv(URL_EMISSION_IMPACTS, sep=";", index_col=0, skiprows=[1])
            self.df_engine_data = pd.read_csv(URL_ENGINE_DATA, sep=";", index_col=0).dropna(axis=1, how='all')
            
            # FIXED: Added encoding='utf-8-sig' to handle potential Byte Order Marks (BOM)
            # which can cause KeyErrors with invisible characters.
            df_assumptions_temp = pd.read_csv(
                URL_ASSUMPTIONS,
                sep=";",
                index_col=0,
                header=None,
                encoding='utf-8-sig'
            ).squeeze("columns")
            df_assumptions_temp.index = df_assumptions_temp.index.str.strip()
            self.df_assumptions = df_assumptions_temp.to_dict()

            self.df_emissions = pd.read_csv(URL_EMISSIONS, sep=";", index_col=0).dropna(axis=1, how='all')
            self._clean_data()
            self.impact_categories = self.df_dual_fuel_impacts.columns.tolist()

        except Exception as e:
            raise RuntimeError(f"Fehler beim Laden der Daten aus GitHub: {e}")

    def _clean_data(self):
        """Cleans column and index names in the DataFrames."""
        for df in [self.df_dual_fuel_impacts, self.df_emission_impacts, self.df_engine_data, self.df_emissions]:
            df.columns = df.columns.str.strip()
            df.index = df.index.map(lambda x: x.strip() if isinstance(x, str) else x)
        
        # Clean MultiIndex columns for background data
        self.df_dual_fuel_background.columns = pd.MultiIndex.from_tuples(
            [(str(col[0]).strip(), str(col[1]).strip()) for col in self.df_dual_fuel_background.columns]
        )

    def find_nearest_load(self, target_load, load_array):
        """Finds the nearest value in a given array."""
        idx = (np.abs(np.array(load_array) - target_load)).argmin()
        return load_array[idx]

    def run_calculation(self, initial_gas_volume, initial_methane_content, years):
        # --- 1. Select Engine ---
        possible_engines = self.df_engine_data[
            (self.df_engine_data['Min. volume flow [m^3/h]'] <= initial_gas_volume) &
            (self.df_engine_data['Max. volume flow [m^3/h]'] >= initial_gas_volume)
        ]
        if possible_engines.empty:
            raise ValueError("Kein passender Motor für die angegebene Gasmenge gefunden.")
        selected_engine_size = possible_engines.index[0]
        engine_specs = possible_engines.loc[selected_engine_size]

        # --- 2. Gas Development & Engine Load Calculation ---
        k_wert = self.df_assumptions['k-Wert [1/a]']
        annual_runtime = self.df_assumptions['Annual runtime [h]']
        calorific_ch4 = self.df_assumptions['Calorific Value CH4 [kWh/m^3]']
        
        gas_dev_data = []
        
        for year in range(1, years + 1):
            gas_volume_t = initial_gas_volume * np.exp(-k_wert * (year - 1))
            methane_content_t = initial_methane_content # Assuming constant for now
            
            energy_produced_kwh = gas_volume_t * (methane_content_t / 100) * calorific_ch4 * (engine_specs['Engine efficency']/100)
            power_kw = energy_produced_kwh
            engine_load = round((power_kw / selected_engine_size) * 100)
            engine_load = max(engine_specs['Minimum Engine Load'], min(100, engine_load))
            
            gas_dev_data.append({
                "Jahr": year,
                "Gasvolumen [m³/h]": gas_volume_t,
                "Methangehalt [%]": methane_content_t,
                "Motorleistung [kW]": power_kw,
                "Motorauslastung [%]": engine_load
            })
        
        df_gas_dev = pd.DataFrame(gas_dev_data)

        # --- 3. Calculate impacts ---
        yearly_impacts = {y: {} for y in range(1, years + 1)}
        
        # Engine Production (Year 1 only)
        prod_factor = engine_specs['Production Factor']
        yearly_impacts[1]['Engine Production'] = self.df_dual_fuel_impacts.loc['Engine Production'] * prod_factor

        for _, year_data in df_gas_dev.iterrows():
            year = int(year_data['Jahr'])
            load = year_data['Motorauslastung [%]']
            nearest_load = self.find_nearest_load(load, self.df_emissions.index)
            
            # Diesel, Oil, Maintenance Impacts
            diesel_consumption_l_h = self.df_dual_fuel_background.loc[nearest_load, (str(selected_engine_size), 'Diesel Consumption [L/h]')]
            total_diesel_l_year = diesel_consumption_l_h * annual_runtime
            
            yearly_impacts[year]['Diesel Consumption'] = self.df_dual_fuel_impacts.loc['Diesel Consumption'] * total_diesel_l_year
            
            yearly_impacts[year]['Oil Consumption'] = self.df_dual_fuel_impacts.loc['Oil Consumption'] * engine_specs['Oil tank volume']
            yearly_impacts[year]['Maintenance'] = self.df_dual_fuel_impacts.loc['Maintenance']

            # Emission Impacts
            total_kwh_year = year_data['Motorleistung [kW]'] * annual_runtime
            emissions_per_kwh = self.df_emissions.loc[nearest_load]
            total_emissions = emissions_per_kwh * total_kwh_year
            
            emission_impacts = self.df_emission_impacts.T.dot(total_emissions.reindex(self.df_emission_impacts.index, fill_value=0))
            yearly_impacts[year]['Emissions'] = emission_impacts
        
        # --- 4. Aggregate totals ---
        total_impacts_by_source = pd.DataFrame(columns=self.impact_categories)
        for year, sources in yearly_impacts.items():
            for source, impacts in sources.items():
                if source not in total_impacts_by_source.index:
                    total_impacts_by_source.loc[source] = pd.Series(0.0, index=self.impact_categories)
                total_impacts_by_source.loc[source] += impacts.fillna(0)
        
        # --- 5. Verification data ---
        engine_loads_all_engines = []
        for engine_size in self.df_engine_data.index:
             engine_specs_ver = self.df_engine_data.loc[engine_size]
             power_kw_ver = df_gas_dev['Gasvolumen [m³/h]'] * (df_gas_dev['Methangehalt [%]'] / 100) * calorific_ch4 * (engine_specs_ver['Engine efficency']/100)
             load_ver = np.clip((power_kw_ver / engine_size) * 100, engine_specs_ver['Minimum Engine Load'], 100)
             engine_loads_all_engines.append({"Motor [kWel]": engine_size, **{f"Jahr {i+1}": l for i, l in enumerate(load_ver)}})

        return {
            "selected_engine": f"{selected_engine_size} kWel",
            "all_impact_categories": self.impact_categories,
            "all_source_categories": sorted(total_impacts_by_source.index.tolist()),
            "total_impacts_by_source": total_impacts_by_source,
            "yearly_impacts": yearly_impacts,
            "verification_gas_dev": df_gas_dev.set_index('Jahr'),
            "verification_engine_load": pd.DataFrame(engine_loads_all_engines).set_index('Motor [kWel]')
        }

# ----------------------------------------------------------------
# GUI class with Tkinter
# ----------------------------------------------------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Ökobilanz-Rechner für Deponiegasverwertung")
        self.geometry("1400x900")
        self.config(bg="#f0f0f0")

        try:
            self.calculator = LCA_Calculator()
        except Exception as e:
            messagebox.showerror("Fehler beim Initialisieren", f"Die Anwendung konnte nicht gestartet werden.\n\n{e}")
            self.destroy(); return

        self._setup_gui()

    def _setup_gui(self):
        # --- Main structure ---
        header_frame = ttk.Frame(self, padding=(20, 10))
        header_frame.pack(fill="x")
        ttk.Label(header_frame, text="Ökobilanz-Rechner", font=("Arial", 24, "bold")).pack()

        input_frame = ttk.Frame(self, padding=(20, 10), relief="groove")
        input_frame.pack(fill="x", padx=10, pady=5)
        self._setup_inputs(input_frame)

        self.notebook = ttk.Notebook(self)
        self.notebook.pack(expand=True, fill="both", padx=10, pady=10)

        # --- Tabs ---
        self.results_tab = ttk.Frame(self.notebook, padding=10)
        self.verification_tab = ttk.Frame(self.notebook, padding=10)
        self.notebook.add(self.results_tab, text="  Ergebnisse  ")
        self.notebook.add(self.verification_tab, text="  Überprüfung  ")

        self._setup_results_tab()
        self._setup_verification_tab()

    def _setup_inputs(self, parent):
        ttk.Label(parent, text="Gasmenge [m³/h]:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.gas_volume_entry = ttk.Entry(parent, width=15); self.gas_volume_entry.grid(row=0, column=1, padx=5, pady=5)
        self.gas_volume_entry.insert(0, "450")

        ttk.Label(parent, text="Methangehalt [%]:").grid(row=0, column=2, padx=15, pady=5, sticky="w")
        self.methane_content_entry = ttk.Entry(parent, width=15); self.methane_content_entry.grid(row=0, column=3, padx=5, pady=5)
        self.methane_content_entry.insert(0, "55")

        ttk.Label(parent, text="Zeitraum [Jahre]:").grid(row=0, column=4, padx=15, pady=5, sticky="w")
        self.years_entry = ttk.Entry(parent, width=15); self.years_entry.grid(row=0, column=5, padx=5, pady=5)
        self.years_entry.insert(0, "10")

        ttk.Button(parent, text="Berechnung starten", command=self._run_calculation_from_gui).grid(row=0, column=6, padx=25, pady=5)

    def _setup_results_tab(self):
        # --- Top sentence ---
        self.selected_engine_label = ttk.Label(self.results_tab, text="", font=("Arial", 14, "italic"))
        self.selected_engine_label.pack(anchor="w", pady=(0, 10))
        
        # --- Total results table ---
        ttk.Label(self.results_tab, text="Gesamtergebnisse (alle Jahre)", font=("Arial", 12, "bold")).pack(anchor="w", pady=(5,5))
        self.total_results_tree = self._create_treeview(self.results_tab, height=8)

        # --- Bar chart section ---
        chart_frame = ttk.Frame(self.results_tab)
        chart_frame.pack(fill="both", expand=True, pady=10)
        ttk.Label(chart_frame, text="Vergleich der Kategorien", font=("Arial", 12, "bold")).pack(anchor="w")
        
        chart_control_frame = ttk.Frame(chart_frame)
        chart_control_frame.pack(fill="x", pady=5)
        ttk.Label(chart_control_frame, text="Wirkungskategorie auswählen:").pack(side="left", padx=(0, 10))
        self.chart_impact_combo = ttk.Combobox(chart_control_frame, state="readonly", width=40)
        self.chart_impact_combo.pack(side="left")
        self.chart_impact_combo.bind("<<ComboboxSelected>>", self._update_chart)

        self.fig = Figure(figsize=(10, 4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=chart_frame)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # --- Specific result section ---
        spec_frame = ttk.Frame(self.results_tab)
        spec_frame.pack(fill="x", pady=10)
        ttk.Label(spec_frame, text="Spezifisches Ergebnis abfragen", font=("Arial", 12, "bold")).pack(anchor="w", pady=(0, 5))
        
        controls = ttk.Frame(spec_frame)
        controls.pack(fill="x")
        self.spec_cat_combo = self._create_spec_combo(controls, "Kategorie:", 0)
        self.spec_year_combo = self._create_spec_combo(controls, "Jahr:", 2)
        self.spec_impact_combo = self._create_spec_combo(controls, "Wirkungskategorie:", 4, width=30)

        self.specific_result_label = ttk.Label(spec_frame, text="Ergebnis: -", font=("Arial", 10, "bold"))
        self.specific_result_label.pack(anchor="w", pady=5)
        
    def _create_spec_combo(self, parent, label, col, width=20):
        ttk.Label(parent, text=label).grid(row=0, column=col, padx=(10, 2), sticky="w")
        combo = ttk.Combobox(parent, state="readonly", width=width)
        combo.grid(row=0, column=col + 1, sticky="w")
        combo.bind("<<ComboboxSelected>>", self._update_specific_result)
        return combo

    def _setup_verification_tab(self):
        ttk.Label(self.verification_tab, text="Deponiegasentwicklung & Motorauslastung", font=("Arial", 12, "bold")).pack(anchor="w", pady=(0, 5))
        self.gas_dev_tree = self._create_treeview(self.verification_tab, height=12)

        ttk.Label(self.verification_tab, text="Maschinenauslastung der verschiedenen Motortypen [%]", font=("Arial", 12, "bold")).pack(anchor="w", pady=(15, 5))
        self.engine_load_tree = self._create_treeview(self.verification_tab, height=6)

    def _create_treeview(self, parent, height):
        frame = ttk.Frame(parent)
        frame.pack(fill="both", expand=True)
        tree = ttk.Treeview(frame, height=height, show="headings")
        vsb = ttk.Scrollbar(frame, orient="vertical", command=tree.yview)
        hsb = ttk.Scrollbar(frame, orient="horizontal", command=tree.xview)
        tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
        vsb.pack(side="right", fill="y")
        hsb.pack(side="bottom", fill="x")
        tree.pack(side="left", fill="both", expand=True)
        return tree

    def _populate_treeview(self, tree, df):
        for item in tree.get_children():
            tree.delete(item)
        
        formatted_df = df.copy()
        for col in formatted_df.columns:
            if pd.api.types.is_numeric_dtype(formatted_df[col]):
                formatted_df[col] = formatted_df[col].apply(lambda x: f"{x:,.2f}")

        tree["columns"] = [df.index.name] + df.columns.tolist()
        for col in tree["columns"]:
            tree.heading(col, text=col)
            tree.column(col, width=150, anchor="e")

        for index, row in formatted_df.iterrows():
            formatted_index = f"{index:,.0f}" if pd.api.types.is_number(index) else index
            tree.insert("", "end", values=[formatted_index] + row.tolist())

    def _run_calculation_from_gui(self):
        try:
            gas_vol = float(self.gas_volume_entry.get())
            meth_cont = float(self.methane_content_entry.get())
            years = int(self.years_entry.get())

            self.results = self.calculator.run_calculation(gas_vol, meth_cont, years)
            
            # --- Populate Results Tab ---
            self.selected_engine_label.config(text=f"Gewählte Motorgröße für die Berechnung: {self.results['selected_engine']}")
            
            total_impacts = self.results['total_impacts_by_source'].sum().to_frame(name="Gesamtwert")
            self._populate_treeview(self.total_results_tree, total_impacts.T)

            self.chart_impact_combo['values'] = self.results['all_impact_categories']
            self.chart_impact_combo.set(self.results['all_impact_categories'][0])
            self._update_chart()

            self.spec_cat_combo['values'] = self.results['all_source_categories']
            self.spec_year_combo['values'] = list(self.results['yearly_impacts'].keys())
            self.spec_impact_combo['values'] = self.results['all_impact_categories']
            self.spec_cat_combo.set(self.results['all_source_categories'][0])
            self.spec_year_combo.set(1)
            self.spec_impact_combo.set(self.results['all_impact_categories'][0])
            self._update_specific_result()
            
            # --- Populate Verification Tab ---
            # FIXED: Corrected argument order for _populate_treeview calls
            self._populate_treeview(self.gas_dev_tree, self.results['verification_gas_dev'])
            self._populate_treeview(self.engine_load_tree, self.results['verification_engine_load'])
            
            self.notebook.select(self.results_tab)

        except ValueError as e:
            messagebox.showerror("Eingabe- oder Berechnungsfehler", str(e))
        except Exception as e:
            messagebox.showerror("Unerwarteter Fehler", f"Ein Fehler ist aufgetreten:\n\n{e}")

    def _update_chart(self, event=None):
        selected_impact = self.chart_impact_combo.get()
        if not selected_impact or not hasattr(self, 'results'): return

        data = self.results['total_impacts_by_source'][selected_impact]
        self.ax.clear()
        data.plot(kind='bar', ax=self.ax)
        self.ax.set_title(f"Beiträge zu: {selected_impact}", fontsize=10)
        self.ax.set_ylabel("Auswirkung")
        self.ax.tick_params(axis='x', rotation=45, labelsize=8)
        self.ax.get_yaxis().set_major_formatter(mticker.FuncFormatter(lambda x, p: f"{x:,.0f}"))
        self.fig.tight_layout()
        self.canvas.draw()

    def _update_specific_result(self, event=None):
        cat = self.spec_cat_combo.get()
        year_str = self.spec_year_combo.get()
        impact = self.spec_impact_combo.get()
        
        if not all([cat, year_str, impact]) or not hasattr(self, 'results'):
            self.specific_result_label.config(text="Ergebnis: -")
            return
        
        year = int(year_str)
        try:
            value = self.results['yearly_impacts'][year][cat][impact]
            self.specific_result_label.config(text=f"Ergebnis: {value:,.4f}")
        except KeyError:
            self.specific_result_label.config(text="Ergebnis: 0.0 (kein Beitrag in diesem Jahr)")
        except Exception:
             self.specific_result_label.config(text="Ergebnis: -")

# ----------------------------------------------------------------
# Main entry point
# ----------------------------------------------------------------
if __name__ == "__main__":
    app = App()
    app.mainloop()

